# Speed Dating

## Challenge description

We will start a new data visualization and exploration project. Your goal will be to try to understand *love*! It's a very complicated subject so we've simplified it. Your goal is going to be to understand what happens during a speed dating and especially to understand what will influence the obtaining of a **second date**.

This is a Kaggle competition on which you can find more details here :

[Speed Dating Dataset](https://www.kaggle.com/annavictoria/speed-dating-experiment#Speed%20Dating%20Data%20Key.doc)

Take some time to read the description of the challenge and try to understand each of the variables in the dataset. Help yourself with this from the document : *Speed Dating - Variable Description.md*

### Rendering

To be successful in this project, you will need to do a descriptive analysis of the main factors that influence getting a second appointment. 

Over the next few days, you'll learn how to use python libraries like seaborn, plotly and bokeh to produce data visualizations that highlight relevant facts about the dataset.

For today, you can start exploring the dataset with pandas to extract some statistics.

In [1]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt

In [143]:
df = pd.read_csv("data-sd.csv", encoding="ISO-8859-1")

In [4]:
df.shape

(8378, 195)

In [5]:
print("Nombre de lignes :", df.shape[0])
print("Nombre de colonnes :", df.shape[1])

Nombre de lignes : 8378
Nombre de colonnes : 195


Au vu du nombre de colonnes, il est préférable de se baser dès le début sur le fichier .doc de description du dataset

In [6]:
print(df["iid"].nunique(), 'personnes uniques ont réalisés', df.shape[0] / 2, 'dates')
# division par 2 car un date comprends 2 personnes 

551 personnes uniques ont réalisés 4189.0 dates


In [7]:
print(df.query('gender==0')["iid"].nunique(), 'femmes')
print(df.query('gender==1')["iid"].nunique(), 'hommes')

274 femmes
277 hommes


In [8]:
print("Répartition des matchs en %")
df['match'].value_counts() / df.shape[0] * 100

Répartition des matchs en %


0    83.528288
1    16.471712
Name: match, dtype: float64

In [9]:
px.pie(df["match"].map({0:"Match", 1:"Pas match"}),'match', title='Répartition des match en %')

In [10]:
print('Nombres de personnes dans la wave 1 :', df.query("wave==1")['iid'].nunique())

Nombres de personnes dans la wave 1 : 20


Comme indiqué dans le doc, dans la wave 1, il y a bien 10 + 10 = 20 personnes

**Observons la wave 1 pour comprendre comment les rencontres sont organisées :**

In [11]:
df_wave1 = df.query("wave == 1").loc[:,["iid", "order", "pid", "gender", "match"]]
df_wave1.head()

,iid,order,pid,gender,match
0,1,4,11.0,0,0
1,1,3,12.0,0,0
2,1,10,13.0,0,1
3,1,5,14.0,0,1
4,1,7,15.0,0,1


Cela signifie que la fille iid 1 (id unique pour toutes les waves) a rencontré un garçon iid 14 au 5ème date de la wave n°1. Vérifions qu'on a la donnée inverse. En se basant sur l'iid 14 :

In [12]:
df_wave1.query("iid == 14") # ici, on aura les résultats des rencontres de la soirée de iid 14

,iid,order,pid,gender,match
130,14,5,1.0,1,1
131,14,1,2.0,1,1
132,14,7,3.0,1,0
133,14,4,4.0,1,1
134,14,2,5.0,1,1
135,14,6,6.0,1,1
136,14,3,7.0,1,1
137,14,8,8.0,1,1
138,14,9,9.0,1,1
139,14,10,10.0,1,0


C'est confirmé. 14 a rencontré 1 au 5ème date de la soirée et c'est bien un garçon. Et ... il y a un match entre les deux !

### Stats générales sur la population

In [13]:
# On mappe l'id de carrière avec l'intitulé indiqué dans le doc Word
df['career_c'] = df['career_c'].fillna(15) # because 15 is Other
dic_career = {1:"Lawyer", 2:"Academic/Research", 3:"Psychologist", 4:"Doctor/Medicine", 5:"Engineer", 6:"Creative Arts/Entertainment",
7:"Banking/Consulting/Finance/Marketing/Business",8:"Real Estate",9:"International/Humanitarian Affairs",10:"Undecided",
11:"Social Work",12:"Speech Pathology",13:"Politics",14:"Pro sports/Athletics",15:"Other",16:"Journalism",17:"Architecture"
}
df['career_title'] = df['career_c'].map(dic_career)

# On mappe l'id d'origine ethnique avec l'intitulé indiqué dans le doc Word
df['race'] = df['race'].fillna(6) # because 6 is Other
dic_race = {1:"Black/African", 2:"European/Caucasian", 3:"Latino/Hispanic", 4:"Asian/Pacific", 5:"Native American", 6:"Other"}
df['race_title'] = df['race'].map(dic_race)

(Pour info, dans le dataset, il n'y pas de 5-Native American, ils sont probablement intégrés dans 2-European/Caucasian, vu la quantité dans la suite de l'analyse)

In [14]:
# On récupère une ligne par iid (par personne) pour faire des stats sur la population générale
df_population = df.groupby(['iid','career_title','race_title','age']).count().reset_index()

In [49]:
df_population.shape

(543, 197)

In [228]:
px.histogram(df_population, y='career_title', orientation='h', title="Répartition des personnes selon la carrière")

Les étudiants et les métiers de la finance/marketing sont les plus représentés

In [17]:
px.histogram(df_population, y='race_title', orientation='h', title="Répartition des personnes selon l'origine ethnique")

Les européens/caucasien sont les plus représentés.

In [18]:
px.histogram(df_population, x='age', orientation='v', title="Répartition des personnes selon l'âge")

In [19]:
print("L'âge moyen est de ", round(df_population['age'].mean(),2), "ans")
print("L'âge médian est de ", round(df_population['age'].median(),2), "ans")
print("L'âge de 95% des personnes sont éloignés de moins de ", round(2*df_population['age'].std(),2), "ans de la moyenne")

L'âge moyen est de  26.36 ans
L'âge médian est de  26.0 ans
L'âge de 95% des personnes sont éloignés de moins de  7.53 ans de la moyenne


Les rencontres se font majoritairement dans des âges proches. On remarque un participant de 55 ans qui pourrait presque être considéré comme outliers. Etant seul, il n'affecte pas trop la distribution.

**La médiane étant quasi identique à la moyenne. La distribution respecte très bien la loi normale.**

Regardons quelles populations sont les plus encluns à faire des matchs :

In [20]:
df_race_match = df.groupby(["race","race_title"]).agg({'match':'sum'}).reset_index() # cette somme est équivalent à count des match=1
df_race_count = df.groupby(["race","race_title"]).agg({'iid':'count'}).reset_index()

In [21]:
df_race_match['match_rate'] = df_race_match['match'] / df_race_count['iid'] * 100

In [22]:
df_race_match.sort_values('match_rate', ascending=False)

,race,race_title,match,match_rate
0,1.0,Black/African,85,20.238095
4,6.0,Other,117,20.000000
2,3.0,Latino/Hispanic,123,18.524096
1,2.0,European/Caucasian,788,16.670193
3,4.0,Asian/Pacific,267,13.471241


La population Black/African va plus souvent faire des matchs avec 20.2% des dates alors que la population Asian/Pacific fera un match dans 13,4% des cas. Comme vu plus haut, le % global de match est de 16.47 %.

In [23]:
# Exactement la même logique que pour "race"
df_career_match = df.groupby(["career_c","career_title"]).agg({'match':'sum'}).reset_index()
df_career_count = df.groupby(["career_c","career_title"]).agg({'iid':'count'}).reset_index()

df_career_match['match_rate'] = df_career_match['match'] / df_career_count['iid'] * 100

df_career_match.sort_values('match_rate', ascending=False)

,career_c,career_title,match,match_rate
13,14.0,Pro sports/Athletics,3,30.000000
2,3.0,Psychologist,54,20.689655
0,1.0,Lawyer,134,19.851852
15,16.0,Journalism,8,18.181818
6,7.0,Banking/Consulting/Finance/Marketing/Business,390,17.972350
3,4.0,Doctor/Medicine,70,16.627078
14,15.0,Other,35,16.279070
10,11.0,Social Work,35,15.909091
5,6.0,Creative Arts/Entertainment,115,15.883978
1,2.0,Academic/Research,360,15.517241


En regardant les catégories suffisemment représentées (il n'y a qu'un seul sportif et un seul architecte par exemple), les Psychologist et Lawyer sont plus encluns à matcher que les International/Humanitarian Affairs et les Engineer.

**Pour créer une fonction plus générique ensuite** : nous avons une colonne "samerace" qui indique si le date était avec des personnes de la même origine ethnique (pas spécialement très éthique cette donnée cependant). On aurait pu le déduire nous même. Et voici comment faire :

In [24]:
# on fait un dataframe où chaque ligne est une personne unique et on indique sont origine ethnique
df_iid = df.groupby(['iid','race']).count().reset_index().loc[:,['iid','race']]
df_iid.head()

,iid,race
0,1,4.0
1,2,2.0
2,3,2.0
3,4,2.0
4,5,2.0


In [25]:
# on fait une jointure vers cette "table" iid pour récupérer la "race" du pid (partner id)
df_same = df.merge(df_iid, how='left', left_on='pid', right_on='iid', suffixes=('', '_partner'))
df_same = df_same.loc[:,['iid','pid','race','race_partner']]
df_same.head()

,iid,pid,race,race_partner
0,1,11.0,4.0,2.0
1,1,12.0,4.0,2.0
2,1,13.0,4.0,4.0
3,1,14.0,4.0,2.0
4,1,15.0,4.0,3.0


In [26]:
df['iid'].isna().sum()

0

In [27]:
# on fait un filtre pour indiquer les même "race" entre les 2 personnes et on peut l'ajouter au df initial
# car aucune row n'a été modifiée ou supprimée grâce au left join (et pas de NaN dans les iid)
df["samerace2"] = df_same['race'] == df_same['race_partner']

In [28]:
df["samerace2"] = df["samerace2"].map({False:0,True:1})
df["race_partner"] = df_same['race_partner'] # autant mettre cette info aussi dans le df principal

In [29]:
# on vérifie que notre calcul correspond bien avec samerace d'origine
(~df["samerace2"] == df["samerace2"]).sum()

0

La somme de ce qui n'est pas identique est égale à 0. **Donc on a réussi à déduire "samerace" !**

Grâce à ça, on peut utiliser cet algo **pour faire une fonction permettant de faire des "same"** sur d'autres champs qui n'existent pas :

In [252]:
def df_add_same(same_field, df_temp):
    df_iid = df.groupby(['iid',same_field]).count().reset_index().loc[:,['iid',same_field]]
    df_same = df_temp.merge(df_iid, how='left', left_on='pid', right_on='iid', suffixes=('', '_partner'))
    df_same = df_same.loc[:,['iid','pid','match',same_field,same_field+'_partner']]
    df_same["same_"+same_field] = df_same[same_field] == df_same[same_field+'_partner']
    df_same["same_"+same_field] = df_same["same_"+same_field].map({False:0,True:1})
    return df_same

Usage avec le type d'études:

In [61]:
df_add_same("field_cd",df.copy()).head()

,iid,pid,match,field_cd,field_cd_partner,same_field_cd
0,1,11.0,0,1.0,8.0,0
1,1,12.0,0,1.0,1.0,1
2,1,13.0,1,1.0,1.0,1
3,1,14.0,1,1.0,1.0,1
4,1,15.0,1,1.0,1.0,1


Usage avec le type de carrière:

In [254]:
df_add_same("career_c",df.dropna(subset="career_c").copy()).head()

,iid,pid,match,career_c,career_c_partner,same_career_c
0,4,11.0,0,1.0,2.0,0
1,4,12.0,0,1.0,1.0,1
2,4,13.0,0,1.0,1.0,1
3,4,14.0,1,1.0,1.0,1
4,4,15.0,0,1.0,1.0,1


### Analyse des rencontres après 3/4 semaines

In [31]:
df_calls = df.groupby(['iid','you_call','them_cal'], dropna=True).count().reset_index().loc[:,['iid','you_call','them_cal']]

In [32]:
total_calls = df_calls.shape[0]
total_you_calls = df_calls.loc[df_calls['you_call'] != 0,'iid'].count()
total_them_cal = df_calls.loc[df_calls['them_cal'] != 0,'iid'].count()

In [33]:
px.histogram(df_calls, x='you_call', orientation='v', title="Nombre de personnes ayant appelé n matchs après 3/4 semaines")

In [34]:
print(round((1 - total_you_calls / total_calls) * 100, 2),"% des gens n'ont jamais rappelé personne")

63.12 % des gens n'ont jamais rappelé personne


In [35]:
px.histogram(df_calls, x='them_cal', orientation='v', title="Nombre de personnes ayant été appelé par n matchs après 3/4 semaines")

In [36]:
print(round((1 - total_them_cal / total_calls) * 100, 2),"% des gens n'ont jamais été rappelé par personne")

52.47 % des gens n'ont jamais été rappelé par personne


In [37]:
df_dates = df.groupby(['iid','date_3'], dropna=True).count().reset_index().loc[:,['iid','date_3']]

In [38]:
total_infos = df_dates.shape[0]
total_dates = df_calls.loc[df_dates['date_3'] != 0,'iid'].count()

In [39]:
print(round((total_dates / total_infos) * 100, 2),"% des gens ont été faire un date avec un de leur match après 3/4 semaines")

35.74 % des gens ont été faire un date avec un de leur match après 3/4 semaines


### Analyse de l'impact de quelques corrélations sur les matchs

**Corrélation des centres d'intérêt des participants sur les matchs:**

In [40]:
df_interest_corr = df.dropna(subset='int_corr')
mean_corr_match = df_interest_corr.loc[df_interest_corr['match'] == 1,'int_corr'].mean()
mean_corr_nomatch = df_interest_corr.loc[df_interest_corr['match'] == 0,'int_corr'].mean()

In [41]:
print("La corrélation moyenne des centres d'intérêt quand il y a eu match est", mean_corr_match)
print("La corrélation moyenne des centres d'intérêt quand il n'y a pas eu match est", mean_corr_nomatch)

La corrélation moyenne des centres d'intérêt quand il y a eu match est 0.21731851851851852
La corrélation moyenne des centres d'intérêt quand il n'y a pas eu match est 0.19182241630276567


On peut conclure qu'il n'y a pas un impact réel des centres d'intérêt sur les matchs

**Corrélation entre la "même race" des participants sur les matchs :**

On recalcule d'abord le % de match sur les dates en général :

In [123]:
count_same = df.shape[0]
count_match = df.query("match == 1").shape[0]
perc_match_global = round((count_match / count_same) * 100, 2)
print("% match au global : ", perc_match_global)

% match au global :  16.47


Puis on calcule le % de match parmi les gens avec la même origine éthnique :

In [188]:
df_same = df.dropna(subset='samerace')
# pour un samerace, quel est mon % de chance d'avoir un match ? pas de match ?
df_same = df_same.query("samerace == 1")
count_same = df_same.shape[0]
count_match = df_same.query("match == 1").shape[0]
perc_match = round((count_match / count_same) * 100, 2)
print("% match avec la même origine ethnique : ", perc_match)

print("Soit",round((perc_match - perc_match_global), 2),"points en + que la moyenne des matchs")

% match avec la même origine ethnique :  17.07
Soit 0.6 points en + que la moyenne des matchs


On regarde ce % selon chaque origine ethnique :

In [247]:
def get_points_bysame(id, type_same, type_init, df_temp):
    try:
        df_same = df_temp.dropna(subset=type_same)
        
        # pour un same, quel est mon % de chance d'avoir un match ? pas de match ?
        df_same = df_same.query(f"{type_same} == 1 & {type_init}==@id")
        count_same = df_same.shape[0]

        count_match = df_same.query("match == 1").shape[0]
        perc_match = round((count_match / count_same) * 100, 2)
        
        return round((perc_match - perc_match_global), 2) # return de la difference
    except:
        pass

In [248]:
results = []
for race_id in range(1,7):
    diff = get_points_bysame(race_id, "samerace", "race", df)
    label_race = dic_race[race_id] # On récupére le label de l'origine dans le dic déjà créé
    print(f"Points de chance en plus d'avoir un match en ayant la même origine {label_race}: ",diff,"")
    results.append(diff)
    
px.bar(x=results, y=[dic_race[race_id] for race_id in range(1,7)], title="Points en plus de la moyenne générale d'avoir un match avec la même origine ethnique")

Points de chance en plus d'avoir un match en ayant la même origine Black/African:  39.09 
Points de chance en plus d'avoir un match en ayant la même origine European/Caucasian:  1.08 
Points de chance en plus d'avoir un match en ayant la même origine Latino/Hispanic:  6.61 
Points de chance en plus d'avoir un match en ayant la même origine Asian/Pacific:  -3.55 
Points de chance en plus d'avoir un match en ayant la même origine Native American:  None 
Points de chance en plus d'avoir un match en ayant la même origine Other:  -6.95 


On observe que les Black/African et Latino/Hispanic, auront plus de chance d'avoir un match s'ils font un date entre eux. Alors que les Asian/Pacific auront plutôt tendance à moins matcher que la moyenne entre eux.

**Corrélation entre le type de carrière des participants sur les matchs :**

On utilise la fonction df_add_same créée précédemment pour fabriquer un dataframe qui contient l'information same_career_c, indiquant si les participants au date avait fait le même type de carrière

In [245]:
df_career = df_add_same("career_c", df.dropna(subset='career_c').copy())

In [251]:
results = []
for field_id in range(1,16):
    diff = get_points_bysame(field_id, "same_career_c", "career_c", df_career)
    label_career = dic_career[field_id] # On récupére le label de l'origine dans le dic déjà créé
    results.append(diff)
    
px.bar(x=results, y=[dic_career[field_id] for field_id in range(1,16)], title="Points en plus de la moyenne générale d'avoir un match avec le même type de carrière")

On voit que les Ingénieur et Psychologues vont avoir bien plus tendance à matcher entre eux.

**Corrélation de la différence d'âge des partipants sur leurs matchs :**

In [44]:
df_diff_age = df.dropna(subset=['age','age_o'])

In [45]:
# on retire les outliers pour ne pas fausser la moyenne. Il y surtout 2/3 âges assez élevé comme on avait vu.
from scipy import stats
import numpy as np
line_count = df_diff_age.shape[0]
df_zscore = stats.zscore(df_diff_age.loc[:,'age'])
df_diff_age = df_diff_age[np.abs(df_zscore) < 3]
df_zscore = stats.zscore(df_diff_age.loc[:,'age_o'])
df_diff_age = df_diff_age[np.abs(df_zscore) < 3]

In [46]:
df_diff_age['diff_age'] = (df_diff_age['age'] - df_diff_age['age_o']).abs()

In [47]:
mean_diffage_match = df_diff_age.loc[df_diff_age['match'] == 1,'diff_age'].mean()
mean_diffage_nomatch = df_diff_age.loc[df_diff_age['match'] == 0,'diff_age'].mean()

In [48]:
print("L'âge moyen quand il y a eu match est", round(mean_diffage_match, 2), 'ans')
print("L'âge moyen quand il y a pas eu match est", round(mean_diffage_nomatch, 2), 'ans')

L'âge moyen quand il y a eu match est 3.06 ans
L'âge moyen quand il y a pas eu match est 3.57 ans


Plus la différence d'âge est élevée, moins il y a de chance qu'il y ait un match.

**En comparant les partenaires entre eux, on peut observer qu'il y a des impacts quand il y a certains points commun.**

-Il y a peu d'influence si les partenaires ont les même centres d'intérêt (en général).

-Peu d'impact aussi s'ils sont Européen/Caucasien.

**Mais 2 partenaires qui sont Black/african et Engineer avec un âge proche auront des très grandes chances de faire un match. Car selon l'origine ethnique et le métier, il a des différences importantes comme nous l'avons vu.**